**Franco Ortiz** :Data scientist

UFO-reports-analysis
# Problem Statement
### Questions to answer
- What time is more frequent to see them?
- How long did the experience take?
- What shapes are most common and where?
- Top words that described that moment (wordcloud)
- heat map with quantities and cities for number of sightings

# Data importing
#### modules

In [6]:
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv("data/ufo.csv")
df.head(10)

,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,00:00,Dieppe (Moncton) (Canada),NB,Canada,Oval,NaN,((HOAX??)) we wher looking .. in the sky in di...,9/24/12,NaN
1,4/ 23:22,Atlantic City,NJ,USA,Disk,30 minutes,Observed a very bright white saucer (had two l...,5/6/17,NaN
2,03142020 23:04,Norwich,VT,USA,Circle,97 seconds,Strange sighting in the sky,5/15/20,NaN
3,1970s 23:00,Grande Summit,PA,USA,Light,5 minutes,4 lights speeding very fast across sky silently,8/16/21,NaN
4,23:00,Desolation Wilderness,CA,USA,Triangle,2 min.,triangle of lights moving slowly through the n...,9/15/05,NaN
5,23:00,Playa del Rey,CA,USA,Fireball,3-5 seconds,"Neon green, long-tailed fireball meteor-lookin...",7/8/04,NaN
6,23:00,Milltown (Canada),NB,Canada,Circle,NaN,"I wonder where they are today, and if they are...",6/4/04,NaN
7,23:00,Manning,SC,USA,Sphere,2 minutes,The object was metallic in color. Spherical in...,6/25/20,NaN
8,23:00,New Alexandria,PA,USA,Other,15 minutes,Translucent UFO.,5/9/19,NaN
9,23:00,"Northern California Coast (in, highway)",CA,USA,Cigar,3-5 min.,We were travelling along the coast of Californ...,10/2/99,NaN


In [ ]:
df.columns

# Data Cleaning

#### date range of timestamps 

In [3]:
print(pd.Timestamp.min)
print(pd.Timestamp.max)

1677-09-21 00:12:43.145224193
2262-04-11 23:47:16.854775807


#### splitting datetime column
outside range is NaT value

In [ ]:
# here i'll be loosing data 
df['Date'] = pd.to_datetime(df['Date / Time'], errors = 'coerce').dt.date
df['Time'] = pd.to_datetime(df['Date / Time'], errors = 'coerce').dt.time
df.head(10)

# HERE not : try strftime
df['Datetime'] = pd.to_datetime(df['Date / Time'])
df['Date'] = df['Datetime'].dt.strftime('%Y/%m/%d')
df['Time'] = df['Datetime'].dt.strftime('%H:%M')

# EDA

# Feature Engineering

# Modelling

# Communication